# Coursera Capstone Project Report

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Introduction

In [ ]:
see Coursera Capstone project PDF a GitHub repository

# Data

In [2]:
# Comparison of the last place visit in London
address = 'MOXY LONDON STRATFORD, LONDON'
geolocator = Nominatim()
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print('The geograpical of MOXY LONDON STRATFORD are {}, {}.'.format(neighborhood_latitude, neighborhood_longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical  of MOXY LONDON STRATFORD are 51.5430646, -0.000851577934734942.


In [3]:

CLIENT_ID = 'R43B4CUXDCBPSKOHNV2ADO1BDBKIJUYDMDXSTBMNCSUL11BT' # your Foursquare ID
CLIENT_SECRET = 'B3GTII5N0DBS0RU1LU20J43Y4C1EGD4EMORZQAHYXL3ZNDN4' # your Foursquare Secret
VERSION = '20191207' # Foursquare API version


In [4]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()
nearby_venues.shape


(63, 4)

In [17]:
nearby_venues.head()

,name,categories,lat,lng
0,M&S Foodhall,Grocery Store,51.542775,-0.004853
1,Theatre Royal Stratford East,Indie Theater,51.542920,0.000779
2,Roof East,Bar,51.541872,-0.001196
3,UNIQLO,Clothing Store,51.542742,-0.004505
4,Pret A Manger,Sandwich Place,51.541476,-0.002399


In [26]:
 
stratford_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=90)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'] +'['+nearby_venues['categories'] +']'):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='Red',
        fill_color='black',
        fill_opacity=0.9,
    ).add_to(stratford_map)  
    
stratford_map